In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.api.types import is_numeric_dtype
sns.set(rc={'figure.dpi':300})

ModuleNotFoundError: No module named 'pandas'

In [9]:

import sklearn

ModuleNotFoundError: No module named 'sklearn'

# Linear regression



<img src="img/linear_regression.png" alt="linear_regression" width="800" style="float: left;"/>

Используя функцию [pd.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) прочтите данные из файла "data/advertising.csv"

In [ ]:
data = pd.read_csv("data/advertising.csv")
data

In [ ]:
target = 'Sales'
features = [c for c in data.columns if c != target]

In [ ]:
sns.pairplot(data, x_vars=..., y_vars=..., height=4, kind='scatter')
plt.show()

Используя функцию [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) библиотеки sklearn, разбейте данные на обучающую и тестовую выборки. Размер тестовой выборки - 20% от всей выборки.

In [ ]:
from sklearn.model_selection import train_test_split
X, Y = data[features], data[target]
X_train, X_test, Y_train, Y_test = ...

Расскоментирйте код внизу и расставьте строки в правильном порядке с тем чтобы успешно обучить модель линейной регрессии и оценить качество предсказания с помощью R<sup>2</sup>-score

In [ ]:
# from sklearn.linear_model import LinearRegression
# r2_score(Y_test, Y_pred)
# model.fit(X_train, Y_train)
# from sklearn.metrics import r2_score
# Y_pred = model.predict(X_test)
# model = LinearRegression()

In [ ]:
low, high = np.min(np.minimum(Y_test, Y_pred)), np.max(np.maximum(Y_test, Y_pred))
ax = sns.scatterplot(x=Y_test, y=Y_pred)
sns.lineplot(x=[low, high], y=[low, high], ax=ax)
ax.set_ylabel(f'Predicted {target}')

# Decision tree and overfitting

<img src="img/dt_meme.jpeg" alt="decision tree" style="float: left;"/>

Прочтите данные из файла "data/sales.csv", выведите несколько первых строчек прочитанной таблицы, используя метод [.head](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html) 

In [ ]:
target = 'IsBadBuy'
features = ['MMRCurrentRetailAveragePrice', 'VehOdo']

## Missing values issue

In [ ]:
data[features + [target]].describe()

Ориентируясь на код предыдущего задания, разбейте датасет на обучающую и тестовую выборку и попрубуйте обучить решающее дерево предсказывать целевой признак

In [ ]:
# code to split the sample

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
# code to train and evaluate the model

Исправьте возникающую ошибку за счет удаления строк, в которых значения признаков или целевой переменной отсутствуют.
Вам поможет следующий код  
`data.dropna(your arguments)`


In [ ]:
# code to drop missing values

## Overfitting

<img src="img/overfit_meme.jpg" alt="overfitting" width="500" style="float: left;"/>

Вновь попробуйте натренировать модель машинного обучения

In [ ]:
# code to split the sample and train and evaluate the model again

Получившееся качество предсказания кажется достаточно высоким. Но является ли полученный результат оптимальным? Проверим это, меняя гиперпараметр классификатора - максимальную глубину решающего дерева - и сравнивая качоство предсказания на обучающей и тестовой выборке для разных значений параметра

In [ ]:
train_q = []
test_q = []
depths = [i for i in range(1, 31)]
early_stop_achieved = False
for d in depths:
    DT = DecisionTreeClassifier(max_depth=d)
    ...  # train the model
    train_q.append(...)  # quality on the train sample
    test_q.append(...)  # quality on the test sample
    print(f'Depth {d}; train acc - {...}, test acc - {...}')

    if len(test_q) > 1 and not early_stop_achieved:
        if test_q[-1] < test_q[-2]:
            print(f'Early stop has been achieved. The optimal depth is {d-1}')
            early_stop_achieved = True

Корректно отрисуйте зависимсоть точности предсказания на обучающей и тестовой выборке от максимальной глубины решающего дерева, используя функцию [sns.lineplot](https://seaborn.pydata.org/generated/seaborn.lineplot.html)

In [ ]:
x_arr = depths
fig, ax = plt.subplots()
sns.lineplot(...)
sns.lineplot(...)
ax.set_ylim(0.5, 1.2)
ax.legend()

Тепловая карта предсказания решающего дерева с оптимальной глубиной

In [ ]:
xname=features[0]; yname=features[1]
model = DecisionTreeClassifier(max_depth=4)

fig, ax = plt.subplots()
model.fit(X_train, Y_train)
print(f'accuracy_score on test: {accuracy_score(Y_test, model.predict(X_test))}')
xlim = (data[xname].min(), data[xname].max())
ylim = (data[yname].min(), data[yname].max())
grid_x = np.linspace(*xlim, 100)
grid_y = np.linspace(*ylim, 100)
xx, yy = np.meshgrid(grid_x, grid_y)
grid = np.c_[xx.ravel(), yy.ravel()]
prediction = model.predict_proba(grid)[:, 1].reshape(xx.shape)
ax.contourf(xx, yy, prediction, cmap=sns.color_palette("rocket", as_cmap=True))

ax = sns.scatterplot(data=X_test.join(Y_test), x=xname, y=yname, hue=target, palette="rocket", ax=ax,
                     marker='.', linewidth=0, s=5)
ax.set_xlim(*xlim)
ax.set_ylim(*ylim)

Тепловая карта предсказания переобученного решающего дерева

In [ ]:
model = DecisionTreeClassifier(max_depth=depths[-1])

fig, ax = plt.subplots()
model.fit(X_train, Y_train)
print(f'accuracy_score on test: {accuracy_score(Y_test, model.predict(X_test))}')
prediction = model.predict_proba(grid)[:, 1].reshape(xx.shape)
ax.contourf(xx, yy, prediction, cmap=sns.color_palette("rocket", as_cmap=True))

ax = sns.scatterplot(data=X_test.join(Y_test), x=xname, y=yname, hue=target, palette="rocket", ax=ax,
                    marker='.', linewidth=0, s=5)
ax.set_xlim(*xlim)
ax.set_ylim(*ylim)

# Feature selection

<img src="img/fs_meme.jpeg" alt="feature selection" style="float: left;"/>

Прочтите и отобразите данные из файла "data/diffusion.csv"

Целевая переменная - "E_regression"

In [ ]:
target = 'E_regression'
sns.histplot(data[target])

In [ ]:
from pandas.api.types import is_numeric_dtype
features = [col for col in data.columns if is_numeric_dtype(data[col])]

Пробуем предсказать целевое свойство методом ближайших соседей

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

Одним из распространенных способов увеличения качества предсказания моделей является уменьшение размерности данных. В частности - предсказание по нескольким наиболее важным признакам. Существует множество способов определения важности признаков, мы воспользуемся самыми быстрыми и простыми из них.  
Отберите 5 наиболее важных признаков с помощью метода `f_regression`. Для отбора признаков полезной будет функция `numpy.argsort`, позволяющая произвести сортировку по ключу

In [ ]:
from sklearn.feature_selection import f_regression
...  # get f_score from X, Y
num_best = 5
best_features_f = np.array(features)[][]  # select best features with key sort

Обновите обучающую и тестовую выборки, вновь натренируйте и оцените модель

In [ ]:
X, Y = data[best_features_f], data[target]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
model.fit(X_train, Y_train)
r2_score(model.predict(X_test), Y_test)

Отберите признаки с помощью mutual_info_regression 

In [ ]:
from sklearn.feature_selection import mutual_info_regression
mi = mutual_info_regression(X, Y)[0]
num_best = 5
best_features_mi = np.array(features)[np.argsort(fscore)][-num_best:]
best_features_mi

In [ ]:
x='Hop activation barrier'
ax = sns.scatterplot(data=data, x=x, y="Site2_AtomicNumber", hue=target)

In [ ]:
ax = sns.boxplot(x="Site1_valence", y=target, data=data)
sns.stripplot(x="Site1_valence", y=target, data=data, color=".3", ax=ax)

Еще раз обновите обучающую и тестовую выборки, натренируйте и оцените модель

# Outliers

<img src="img/outliers_meme.png" alt="outliers" style="float: left;"/>

Прочтите и отобразите данные из файла "data/mobile_price.csv"

In [ ]:
target = 'Price'
features = [col for col in data.columns if col != target and is_numeric_dtype(data[col])]

In [ ]:
X, Y = data[features], data[target]

Натренируйте и оцените ансамблевую модель `ExtraTreesRegressor`.\
На этот раз не используйте стандартную полседовательность `train_test_split` - `fit` - `predict`. Вместо этого воспользуйтесь одной единственной функцией `cross_val_predict`. Сохораните результат работы функции в переменную `Y_pred`.

Диаграмма рассеняния true-predicted

In [ ]:
low, high = np.min(np.minimum(Y, Y_pred)), np.max(np.maximum(Y, Y_pred))
ax = sns.scatterplot(x=Y, y=Y_pred)
sns.lineplot(x=[low, high], y=[low, high], ax=ax)
ax.set_ylabel(f'Predicted {target}')

Равномерно ли распределена величина ошибки среди точек выборки?

In [ ]:
abs_error = np.abs(Y - Y_pred)
argsort = np.argsort(abs_error)[::-1]
sns.lineplot(x=np.arange(Y.shape[0]) + 1, y=abs_error[abs_error.index[argsort]], color='violet')

Отберите все точки датасета кроме 60 с наибольшей ошибкой. Используйте метод датафрейма `.loc`. Вновь сделайте предсказание методом `cross_val_predict`

In [ ]:
less_error_idx = abs_error.index[argsort][60:]
...

In [ ]:
low, high = np.min(np.minimum(Y, Y_pred)), np.max(np.maximum(Y, Y_pred))
ax = sns.scatterplot(x=Y, y=Y_pred)
sns.lineplot(x=[low, high], y=[low, high], ax=ax)
ax.set_ylabel(f'Predicted {target}')

# Feature engineering

Прочтите данные о пассажирах Титаника. Посмотрите на данные. Какой тип данных содержится в каждой колонке? Какие колонки можно сразу использовать в качетсве признаков, а какие не будут приняты моделями в качестве входа?

In [ ]:
target = 'Survived'
features = ['Pclass', 'Name', 'Age', 'Cabin', 'Embarked']

Используя методы `.describe` и `.nunique` выведите статистическую информацию по колонкам датасета. О каких дополнительных сложностях предупреждает полученная информация?

Выведите уникальные значения призака 'Cabin'

## Process Age 

С помощью метода `.fillna` заполните пропущенные значения признака "Age" любым значением меньше 0

In [ ]:
# your line here
data['Age'] = data['Age'].map(lambda x: x // 10 * 10).astype('int')

## Process Cabin

С помощью метода `.fillna` заполните пропущенные значения признака "Cabin"

In [ ]:
# your line here
data['CabinFirstLetter'] = data.Cabin.map(lambda s: s[0])
data['CabinNumber'] = data.Cabin.map(lambda s: ''.join(filter(lambda c: c in '0123456789 ', s)))
data['CabinNumber'] = data.CabinNumber.map(lambda s: s if s else '0').map(lambda s: [w for w in s.split() if w][0]).astype('int')
features.remove('Cabin')
features.extend(('CabinNumber', ))

In [ ]:
sns.catplot(x="CabinFirstLetter", y=target, data=data, kind='bar')

In [ ]:
ax = sns.boxplot(x=target, y="CabinNumber", data=data)
sns.stripplot(x=target, y="CabinNumber", data=data, color=".3", ax=ax)

Какие выводы можно сделать по полученным графикам? Имеется ли зависимость между полученными нами признаками и целевой переменной?

## Process Name

Правильным образом вызовите метод `fit_transform` у объекта `vectorizer` чтобы перевести в числовой формат имена пассажиров "Титаника". После вызова примените метод `.toarray` чтобы получить новую матрицу признаков из текста в правильном формате. Присвойте эту матрицу в переменную `NameFeatures`.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=12, binary=True)
...  # your line here
vectorizer.get_feature_names_out()

Используя метод датафрейма `.join` присоедините матрицу признаков `NameFeatures` к имеющимся данным `data`

In [ ]:
NameFeatures = pd.DataFrame(data=NameFeatures, columns=vectorizer.get_feature_names_out())
NameFeatures.columns = NameFeatures.columns.map(lambda s: f'Name_{s}')
...  # your line here
features.remove('Name')
features.extend(NameFeatures.columns)

## One-hot encoding

In [ ]:
cols_to_one_hot = ('Pclass', 'Embarked', 'Age', 'CabinFirstLetter')
for col in ('Pclass', 'Embarked', 'Age'):
    features.remove(col)

Расскоментирйте код внизу и расставьте строки в правильном порядке с тем чтобы успешно применить one-hot encoding к категориальным признакам из списка

In [ ]:
# data = data.join(oh_data)
# for col in cols_to_one_hot:
# features.extend(oh_cols_train)
# oh_data = pd.DataFrame(index=data.index)
# oh_cols_train = oh_data.columns.to_list()
# data.head()
#     oh_data = oh_data.join(pd.get_dummies(data[col], prefix=col).astype('float'))

## Modeling

Натренируйте модель `LogisticRegressionCV` на полученных признаках

# Design of experiment. Grid, LHS and adaptive sampling

<img src="img/exp_meme.jpg" alt="experiment" style="float: left;"/>

In [ ]:
!/opt/anaconda_py39_1/bin/pip install pyDOE2
!/opt/anaconda_py39_1/bin/pip install --upgrade minlpe

In [ ]:
ndim = 3
dimlims = (-5., 5.)

In [ ]:
def func(x1, x2, x3):
    return 0.2 * x1 ** 3 - 0.5 * x2 ** 2

In [ ]:
fig, ax = plt.subplots()
grid_x = np.linspace(*dimlims, 100)
grid_y = np.linspace(*dimlims, 100)
xx, yy = np.meshgrid(grid_x, grid_y)
points = np.hstack((xx.ravel().reshape(-1, 1), yy.ravel().reshape(-1, 1)))
ax.contourf(xx, yy, func(points[:, 0], points[:, 1], 0).reshape(xx.shape),
           cmap='viridis')

In [ ]:
points_per_dim = 3

In [ ]:
def get_grid(points_per_dim):
    grid = np.hstack(
        [xx.reshape(-1, 1) for xx in np.meshgrid(*[np.linspace(-5., 5., points_per_dim) for _ in range(ndim)])]
    )
    return grid

In [ ]:
grid = ...

In [ ]:
test_size = 200
test_points = 10. * np.random.random(size=(test_size, ndim)) - 5.
test_fvales = func(...)  # evaluate function at test points

Натренируйте ансамблевую модель `ExtraTreesRegressor` на простой сетке точек. Оцените качество на тестовой выборке

In [ ]:
from pyDOE2 import lhs
lhs_points = 10. * lhs(ndim, samples=points_per_dim ** ndim) - 5.
lhs_fvalues = func(...)   # evaluate function at lhs points

Натренируйте ту же модель на наборе точек LHS. Оцените качество на тестовой выборке

In [ ]:
from minlpe import minlpe
adaptive_points, adaptive_fvalues, _ = minlpe(lambda x: func(x[0], x[1], x[2]), [[-5., 5.]] * ndim, maxSampleSize=points_per_dim ** ndim,
                                              folder='adaptive')

Натренируйте модель еще раз на наборе точек полученнных с помощью адаптивного сэмплинга и оцените качество на тестовой выборке

Пример генерации точек адаптивным сэплингом в режиме online.

In [ ]:
from minlpe import nextPoint
samplerParams = dict(seed=0, samplerTimeConstraint=None, debug=False)
X = [[-5.0, -5.0, 0.], [5.0, 5.0, 0.]]
Y = [func(*x) for x in X]
newX1 = nextPoint(X, Y, bounds=[[-5, 5], [-5, 5], [-5, 5]], **samplerParams)
newX1  # новые параметры "эксперимента"

In [ ]:
X.append(newX1)  
Y.append(func(*newX1))  # новое "экспериментальное" значение для параметров предлложенных алгоритмом
newX2 = nextPoint(X, Y, bounds=[[-5, 5], [-5, 5], [-5, 5]], **samplerParams)
newX2

# It is your turn now!

In [ ]:
data =  # read the file "data.competition.csv"
target = 'octamer'
SEED = 123

In [ ]:
...

In [ ]:
cross_val_predict(model, X, Y, ranodm_state=SEED)